In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings("ignore")

In [2]:
names_df = pd.read_csv('./Keyword.csv')
names_df

,0
0,종로구 상해
1,종로구 짱구
2,종로구 교촌치킨 숭인점
3,종로구 수제비집
4,종로구 아람
...,...
665,종로구 속이 알찬 주먹밥
666,종로구 좋아좋아분식
667,종로구 헬로사이공
668,종로구 호랭이 뼈다귀


In [8]:
# 창이 나오지 않게 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')

path = 'chromedriver.exe' # 크롬드라이버 경로 설정

source_url = "https://map.kakao.com/" # 리뷰주소 크롤링을 해 올 주소 설정

page_urls = [] #리뷰주소 크롤링 결과를 담을 리스트

driver = webdriver.Chrome(executable_path=path, chrome_options=options)

for name in tqdm_notebook(names_df['0']):
    try:
        driver.get(source_url)

        search_bar = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창 찾기
        search_bar.send_keys(name)

        searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색 버튼 누르기
        driver.execute_script("arguments[0].click();", searchbutton)

        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="info.search.place.list"]/li[2]/div[3]/strong/a[1]')))

        html = driver.page_source # 페이지 소스 설정

        soup = BeautifulSoup(html, "html.parser") # 페이지 소스 파싱
        moreview = soup.find(name="a", attrs={"class":"moreview"}) # 상세 보기 찾기
    
        page_url = moreview.get("href") # 상세보기에 링크된 리뷰주소 가져오기
        page_urls.append(page_url) 
    except:
        pass

driver.close()

  0%|          | 0/670 [00:00<?, ?it/s]

In [9]:
page_url

'https://place.map.kakao.com/586571741'

In [ ]:
# 이미 리뷰주소가 있을 경우 사용
'''page_urls = []

for i in names_df.리뷰주소:
    page_urls.append(i)

options = webdriver.ChromeOptions()
options.add_argument('headless')

path = 'C:/Users/USER/TIL/chromedriver' '''

In [10]:
columns = ['name', 'score', 'review']
review_df = pd.DataFrame(columns=columns)

driver = webdriver.Chrome(executable_path=path, chrome_options=options)

for page_url in tqdm_notebook(page_urls):
    try:
        driver.get(page_url)
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/h2'))) # 가게 이름이 보일 때까지
        

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        names = soup.find(name='h2', attrs={"class":"tit_location"}) # 가게 이름 찾기
        
        contents_div = soup.find(name="div", attrs={"class":"evaluation_review"}) # 후기 작성란 찾기
        rates = contents_div.find_all(name="em", attrs={"class":"num_rate"}) # 별점 찾기
        reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"}) # 리뷰 찾기
    
        # 첫번째 페이지 별점 및 리뷰 가져오기
        for rate, review in zip(rates, reviews):
            if review != ' ':
                row = [names.text, rate.text[0], review.find(name="span").text] # 가게이름, 별점, 리뷰 순으로 가져와 리스트 만들기
                series = pd.Series(row, index=review_df.columns)
                review_df = review_df.append(series, ignore_index=True)
        
        # 두번째 페이지부터 다섯번째 페이지까지 별점 및 리뷰 가져오기
        for button_num in range(2, 6): 
            another_reviews = driver.find_element_by_xpath("//a[@data-page='" + str(button_num) + "']") # 페이지 넘기기
            another_reviews.click()
            driver.implicitly_wait(5)
            
        
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})
            rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})
            reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})

            for rate, review in zip(rates, reviews):
                if review != ' ':
                    row = [names.text, rate.text[0], review.find(name="span").text]
                    series = pd.Series(row, index=review_df.columns)
                    review_df = review_df.append(series, ignore_index=True)
    except:
        pass # 리뷰가 없거나 리뷰를 공개하지 않은 곳들도 있어서 패스 설정
    
driver.close()

  0%|          | 0/281 [00:00<?, ?it/s]

In [11]:
review_df

,name,score,review


In [6]:
# 리뷰 없이 별점만 남긴 리뷰 제거
filter = review_df["review"] != "" 
review_data = review_df[filter]
review_data

,name,score,review


In [8]:
review_data.to_csv("./review_data.csv", index=False, encoding='utf-8-sig')